<a href="https://colab.research.google.com/github/norahyk/pythonsnippet/blob/main/api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/norahyk/pythonsnippet/blob/master/generate.ipynb)

In [ ]:
import pandas as pd
import requests
import json

In [ ]:
url_fmt = 'https://holidays-jp.github.io/api/v1/{year}/date.json'
year = '2021'
res = requests.get(url_fmt.format(year=year))
holidays = json.loads(res.content)
holidays

{'2021-01-01': '元日',
 '2021-01-11': '成人の日',
 '2021-02-11': '建国記念の日',
 '2021-02-23': '天皇誕生日',
 '2021-03-20': '春分の日',
 '2021-04-29': '昭和の日',
 '2021-05-03': '憲法記念日',
 '2021-05-04': 'みどりの日',
 '2021-05-05': 'こどもの日',
 '2021-07-22': '海の日',
 '2021-07-23': 'スポーツの日',
 '2021-08-08': '山の日',
 '2021-08-09': '休日 山の日',
 '2021-09-20': '敬老の日',
 '2021-09-23': '秋分の日',
 '2021-11-03': '文化の日',
 '2021-11-23': '勤労感謝の日'}

In [ ]:
def make_df(item):
    df = pd.DataFrame(pd.date_range('2021/4/20', periods=30), columns=['salesdate'])
    df['holiday_name'] = df['salesdate'].dt.strftime('%Y-%m-%d').map(holidays)
    df['goodscode'] = item
    return df
df = pd.concat([make_df(g) for g in list('ABC')])

,salesdate,holiday_name
0,2021-04-20,None
1,2021-04-21,None
2,2021-04-22,None
3,2021-04-23,None
4,2021-04-24,None
5,2021-04-25,None
6,2021-04-26,None
7,2021-04-27,None
8,2021-04-28,None
9,2021-04-29,昭和の日


In [ ]:
df = df.resample('D', on='salesdate')[['holiday_name']].first().reset_index()
df['is_holiday'] = ~df['holiday_name'].isna()
df['is_weekend'] = df['salesdate'].astype('datetime64').dt.weekday >= 5
df['is_dayoff'] = df['is_holiday'] | df['is_weekend']

num_of_daysoff_or_not = (df['is_dayoff'] != df['is_dayoff'].shift()).cumsum()
df['whatday_of_the_daysoff'] = df['is_dayoff'].groupby(num_of_daysoff_or_not).cumsum()

df['after_the_daysoff'] = (df['is_dayoff'].shift() * ~df['is_dayoff']).fillna(0).astype('bool')
df

,salesdate,holiday_name,is_holiday,is_weekend,is_dayoff,whatday_of_the_daysoff,after_the_daysoff
0,2021-04-20,None,False,False,False,0,False
1,2021-04-21,None,False,False,False,0,False
2,2021-04-22,None,False,False,False,0,False
3,2021-04-23,None,False,False,False,0,False
4,2021-04-24,None,False,True,True,1,False
5,2021-04-25,None,False,True,True,2,False
6,2021-04-26,None,False,False,False,0,True
7,2021-04-27,None,False,False,False,0,False
8,2021-04-28,None,False,False,False,0,False
9,2021-04-29,昭和の日,True,False,True,1,False
